# 一 远程控制
## 1. subprocess模块应用
我们写好的软件服务端都是部署在服务器上的，而服务器一般都是放在机房，由于成本问题，机房一般都是建在郊区甚至山区，不可能建在闹市区。那么我们需要查看服务器运行状态就需要远程登陆服务器，如果你非的要跑到山区去接显示器和键盘也没人拦着你。

**服务端代码**



In [ ]:
from socket import *
import subprocess
server=socket(AF_INET,SOCK_STREAM)
server.bind(('127.0.0.1',8080))  # 127.0.0.1是本地回环地址，一般用来做测试
server.listen(5)
while True:
    conn,client_address=server.accept() #(连接对象，客户端的ip和端口)
    print(client_address)
    while True:
        try:
            cmd=conn.recv(1024)
            obj=subprocess.Popen(cmd.decode('utf-8'),
                                 shell=True,
                                 stdout=subprocess.PIPE,
                                 stderr=subprocess.PIPE
                                 )
            stdout=obj.stdout.read()
            stderr=obj.stderr.read()
            # 发送真实的数据
            conn.send(stdout)
            conn.send(stderr)
        except ConnectionResetError:
            break
    conn.close()
server.close()

('127.0.0.1', 1952)


## 2. 粘包现象
![avatar](https://cdn.nlark.com/yuque/0/2019/png/281865/1554573280905-0de384fd-adc4-48a6-9149-7ced7dc7f732.png#align=left&display=inline&height=578&originHeight=436&originWidth=720&size=0&status=done&width=955)

![avatar](https://cdn.nlark.com/yuque/0/2019/png/281865/1554573280917-6c0ecbd5-cb13-415f-8929-7ea2e54bcd79.png#align=left&display=inline&height=358&originHeight=519&originWidth=1081&size=0&status=done&width=746)

前面的图是逻辑意义图，而这个图是工作流程图，我们以为的TCP协议是直接发送给对方其实不然，上图绿色和红色分别表示应用程序和操作系统的内存，对于应用程序来说只需要完成send那一行代码之后，就是发送出去了，而其实只是把需要发送的数据复制给了操作系统内存一份，对于操作系统而言，这只是工作的开始，操作系统开始控制传输层和传输层以下下进行发送。
对于接收的一方来说，先到达的也是它的操作系统，如果接收方没有recv或者recv没有接收完全，那么数据就会滞留在接收方的操作系统内，直到下一次发送方发送数据，接收方接收数据就会把残留的结果顶出来，这就是流式协议，也是产生粘包现象的原因。由于操作系统缓存的存在，才会产生上文中的第二次执行ls命令结果不是两部分的现象。

## 4. 解决粘包问题
要解决这个粘包问题的根本就是接收方必须要清楚的知道发送方发了多少数据，进而再进行循环接收，那么这样不可避免的涉及到了自定义报头，这这里补充一个小的知识点struct这个模块的用法。


In [1]:

import struct
res1 = struct.pack('i', 1231)  # 把数字转化成bytes，并且固定长度
res2 = struct.pack('i', 1)
res3 = struct.pack('i', 1473985)
print(res1,len(res1))
print(res2,len(res2))
print(res3,len(res3))
res4 = struct.unpack('i', res3)
print(res4)

b'\xcf\x04\x00\x00' 4
b'\x01\x00\x00\x00' 4
b'\xc1}\x16\x00' 4
(1473985,)


**练习一**

1.    基于TCP协议写一个远程执行命令的程序，要求不能产生任何粘包，recv参数设置成512


In [7]:
#服务端
from socket import *

server = socket(AF_INET, SOCK_STREAM)
server.bind(('127.0.0.1', 8080))
server.listen(5)

conn, client_address = server.accept()

res1 = conn.recv(512)  # 发了五个就收五个
res11 = struct.pack('5s', res1)
print('第一次：', res1)
print('第一次struct：', res11)
res2 = conn.recv(512)
res22 = struct.pack('6s', res2)
print('第二次: ', res2)
print('第二次struct：', res22)
conn.close()
server.close()

第一次： b'hello'
第一次struct： b'hello'
第二次:  b'world'
第二次struct： b'world\x00'


现在你清楚了不管是接收和发送并不是直接和对方对接，而是通过自己的操作系统缓存，最开始我们设定的最大接收的值是1024个字节，如果要发送的数据是一个T，你把这个值也无限调大到一个T，那么，请你先给我造一台内存超过一个T的电脑出来。

涉及到操作系统缓存，其实就像是从大海里面捞鱼，鱼有很多，你不可能造一个无限大的网，但是你可以用一个小网重复捞，循环捞。

# 二 UDP协议通信
## 1. UDP与TCP比较说明
UDP通信不需要建立双向连接，也不需要等ack=1的确认信息，所以它的传输会比TCP快，但是通过UDP通信由发送方传给接收方的数据，一旦在传输过程中丢失，发送方也会把自己的缓存清理掉，而TCP协议则是必须要等到接收方回传ack=1才会清理自己的缓存，如是，UDP快但不安全，TCP慢但是安全，UDP快的主要原因是少了一个确认的回传信息，也就是少了一个网络延迟，而不是他不需要连接（当然这也要花时间），那么同理，TCP比UDP可靠也是因为它的确认回传信息，而不是它的双向连接。

## 2. UDP套接字
UDP通信不需要建立连接，但是服务端依然是需要绑定ip和端口的，程序启动的时候也可以先启动客户端，客户端不管服务端是否接收了，发了就完事。


服务端代码：


In [ ]:
import socket
server=socket.socket(socket.AF_INET,socket.SOCK_DGRAM) # 数据报协议
server.bind(('127.0.0.1',8080))
while True:
    client_data,client_addr=server.recvfrom(1024)
    msg=input('回复%s:%s>>>:hello' %(client_addr[0],client_addr[1]))
    server.sendto(msg.encode('utf-8'),client_addr)

In [ ]:
##客户端代码：

import socket
client=socket.socket(socket.AF_INET,socket.SOCK_DGRAM) # 数据报协议
while True:
    msg=input('>>>: ').strip()
    client.sendto(msg.encode('utf-8'),('127.0.0.1',8080))
    res,server_addr=client.recvfrom(1024)
    print(res.decode('utf-8'))

你可以把以上客户端代码复制多份，同时向服务端发送消息，你可以看到UDP通信看似可以同时服务于多个客户端，那只是因为他不需要连接服务的速度很快，快到你感知不出来，像这种同时服务于多个客户端快到我们感知不出来的情况，这就叫做并发。如果你把客户端代码复制100万份，使用100万台电脑和100万个人同时向服务端发送消息，那么不同客户端收到消息是一定会有先后顺序的，因为说到底，服务端终究只是一个人在服务，就像我们的TCP通信的服务端一样，客户端少了你自然感知不出来这个时间差，而当客户端多了之后，服务端的服务是一定会有一个先后顺序的，如何解决这个问题呢？就说在服务端造出来多个人来提供服务。假如服务端有100万个人提供服务，同时向服务端发送消息的客户端也要有100万个，那么一个萝卜一个坑，这个就是真正的服务端同时服务于多个客户端了，这就叫做并行。但实际上，假如一开一个饭店，每天同时来的客人有100位，你会找了100为服务员？除非你人傻钱多，否则绝对不会的，我们可以让你个服务员轮流服务客人，只要能够运转开就可以了。同理，服务员就是你的机器的硬件。
## 3. UDP协议套接字的特点
UDP协议通信一个发送对应一个接收，不会产生粘包问题，因为操作系统监测到你是用数据报协议发送数据会自动定义报头，以此来保证一个发送对应一个接收。
验证UDP协议客户端不粘包



In [ ]:
import socket
server=socket.socket(socket.AF_INET,socket.SOCK_DGRAM) # 数据报协议
server.bind(('127.0.0.1',8080))
res1,client_addr=server.recvfrom(512) 
print(res1)
res2,client_addr=server.recvfrom(512) 
print(res2)
res3,client_addr=server.recvfrom(512) 
print(res3)

In [ ]:
# 客户端代码

import socket
client=socket.socket(socket.AF_INET,socket.SOCK_DGRAM) # 数据报协议
client.sendto(b'hello',('127.0.0.1',8080))
client.sendto(b'world',('127.0.0.1',8080))
client.sendto(b'albert',('127.0.0.1',8080))
# 验证UDP协议服务端不粘包
# 服务端代码修改如下（客户端代码不变）

import socket
server=socket.socket(socket.AF_INET,socket.SOCK_DGRAM) # 数据报协议
server.bind(('127.0.0.1',8080))
res1,client_addr=server.recvfrom(1)  # b‘h‘ 剩下的就不要了，不会粘到下一次接收
print(res1)
res2,client_addr=server.recvfrom(2)   # b‘wo‘
print(res2)
res3,client_addr=server.recvfrom(3)   # b‘alb‘
print(res3)

##  4. UDP协议套接字应用
我们知道浏览器是一个套接字客户端软件，我们如果需要上网的话，只需要输入被访问的网址链接就可以了。我们以 “> https://zhuanlan.zhihu.com/p/54491788“ 这个链接为例，这是一个url地址，叫统一资源定位符，用来定位互联网上独一无二的一个资源，其实就是一份数据或者一个文件。 url地址分为三部分 （1）https://（2）"> zhuanlan.zhihu.com"（3）/p/54491788第一部分是应用层的协议，第二部分是域名，第三部分是文件路径，我们浏览器的其中一个功能就是把这个文件下载到本地，给用户观看。浏览器需要找到服务端主机在哪里，找了服务端主机之后，服务端主机一行一行的读取文件内容，在发送给客户端浏览器，这就是最基本的上过的过程。
那么问题是客户端软件怎么找到服务端软件，自然是通过ip和端口，但是我们现在看到的只有一个域名和服务端主机有关联，域名能帮我们定位到服务端在哪里吗？那么必然会发生的事情就是这个这个域名能够转化的ip和端口，其实这个域名后面省略了一个东西就是“:80”，这个端口指的是服务端的端口，所以我们访问 https://zhuanlan.zhihu.com:80/p/54491788
也是一样的。既然已经有了默认的端口，那么必然有一个软件将这个域名解析成一个ip地址，这个软件的客户端是你机器上面自带的，叫做DNS。由于一个域名和一个ip地址这样的数据量远没有达到512个字节，为了是通信更迅速，我们使用UDP协议，这个DNS就是基于UDP协议通信的。
![avatar](https://cdn.nlark.com/yuque/0/2019/png/281865/1554573280969-6d5a1fa9-addf-4323-a1e4-341fa361203c.png#align=left&display=inline&height=535&originHeight=451&originWidth=720&size=0&status=done&width=854)


**练习2**

2.    基于UDP写一个qq聊天程序，要求使用软件开发规范，用户可以注册qq，登陆qq，可以多人群聊，也可以一对一聊天
想法分析：因为udp不需要建立双向连接，所以我们只需要绑定ip和端口就可以接发信息

In [ ]:
import socket
server=socket.socket(socket.AF_INET,socket.SOCK_DGRAM) # 数据报协议
server.bind(('127.0.0.1',8080))
while True:
    res1,client_addr=server.recvfrom(512)  
    print(res1)
   

b'hello'
b'world'


**练习3**
基于UDP协议写一个NTP程序（这是集群必须要做的事情，以后你们有机会接触集群就知道了）

In [ ]:
from socket import *
import time
ip_port=('127.0.0.1',8085)
buffer_size=1024
udp_server=socket(AF_INET,SOCK_DGRAM)
udp_server.bind(ip_port)

while True:
    fmt='%Y-%m-%d %X'
    data,addr=udp_server.recvfrom(buffer_size) #返回的是元组
    print('来自客户端：',data)
    server_time=time.strftime(fmt)
    udp_server.sendto(server_time.encode('utf-8'),addr)

来自客户端： b'123'


In [8]:
import hashlib   
src = 'this is a md5 test.' 
m2 = hashlib.md5()   
m2.update(src.encode('utf-8'))   
print (m2.hexdigest())

174b086fc6358db6154bd951a8947837
